In [54]:
import sys
import random
from collections import deque

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle5

%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../src')
import const
from utils import DataHandler

In [55]:
dh = DataHandler()

In [56]:
nrows = 30_000_000
train_df = pd.read_csv('../data/input/train.csv', dtype=const.DTYPE, nrows=nrows)
# val_idx = np.load('../data/processed/cv1_valid.npy')

# train_df = train_df.iloc[val_idx[np.where(val_idx < nrows)]]
# exp_test_df = train_df.groupby('user_id').tail(6)

# exp_test_df['group_num'] = 0

# for user_id, user_df in exp_test_df.groupby('user_id'):
#     user_idx = user_df.index
    
#     prior_task_id = -1
#     group_num = 0
#     user_group_array = np.zeros(len(user_df))
#     for idx, task in enumerate(user_df['task_container_id']):
#         if prior_task_id != task and idx > 0:
#             group_num += 1
        
#         user_group_array[idx] = group_num
#         prior_task_id = task
        
#     exp_test_df.loc[user_idx, 'group_num'] = user_group_array
    
# exp_test_df['group_num'] = exp_test_df['group_num'].astype(int)

# gp = exp_test_df.groupby(['user_id', 'group_num'])

# prior_target_dict = {}
# for (user_id, group_num), df in gp:
#     user_group_id = str(user_id) + '__' + str(group_num + 1)
#     prior_target_dict[user_group_id] = df['answered_correctly'].tolist()
    
# exp_test_df['user_group_id'] = exp_test_df['user_id'].astype(str) + '__' + exp_test_df['group_num'].astype(str)
# exp_test_df['prior_group_answers_correct'] = exp_test_df['user_group_id'].map(prior_target_dict)

# exp_test_df['user_task_count'] = gp.cumcount()

# nan_idx = exp_test_df[exp_test_df['user_task_count'] > 0].index
# exp_test_df.loc[nan_idx, 'prior_group_answers_correct'] = np.nan

# exp_test_df = exp_test_df.drop(['user_group_id', 'user_task_count'], axis=1)

exp_test_df = pd.read_csv('../data/input/example_test.csv')
# sub_df = pd.read_csv('../data/team/sub_check.csv')

In [4]:
exp_test_df.head(10)

,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN
5,5,0,18020362258,1364159702,12023,0,4424,18000.0,True,NaN,NaN
6,6,0,2325432079,1521618396,574,0,1367,18000.0,True,NaN,NaN
7,7,0,39456940781,1317245193,12043,0,5314,17000.0,True,NaN,NaN
8,8,0,3460555189,1700555100,7910,0,532,21000.0,True,NaN,NaN
9,9,0,2214770464,998511398,7908,0,393,21000.0,True,NaN,NaN


In [5]:
with open('../data/team/riiidFE.pkl', mode='rb') as f:
    riiidFE = pickle5.load(f)

In [6]:
cfg = {'compe': {'name': 'riiid-test-answer-prediction'},
 'common': {'seed': 2020,
  'metrics': {'name': 'auc', 'params': {}},
  'drop': ['lecture_idx'],
  'debug': False,
  'kaggle': {'data': False, 'notebook': False}},
 'model': {'backbone': 'transformer_saint_v6_2',
  'n_classes': 1,
  'epochs': 30,
  'params': {'dim_model': 256,
   'num_en': 2,
   'num_de': 2,
   'heads_en': 8,
   'heads_de': 8,
   'total_ex': 13523,
   'total_cat': 7,
   'total_tg': 188,
   'total_in': 2,
   'total_exp': 2,
   'seq_len': 121},
  'multi_gpu': True,
  'head': None},
 'data': {'train': {'dataset_type': 'CustomTrainDataset7_2',
   'is_train': True,
   'params': {'n_skill': 13523, 'max_seq': 121},
   'loader': {'shuffle': True, 'batch_size': 512, 'num_workers': 4},
   'transforms': None},
  'valid': {'dataset_type': 'CustomTestDataset7_2',
   'is_train': False,
   'params': {'n_skill': 13523, 'max_seq': 121},
   'loader': {'shuffle': False, 'batch_size': 512, 'num_workers': 4},
   'transforms': None},
  'test': {'dataset_type': 'CustomTestDataset7_2',
   'is_train': False,
   'params': {'n_skill': 13523, 'max_seq': 121},
   'loader': {'shuffle': False, 'batch_size': 512, 'num_workers': 4},
   'transforms': None}},
 'loss': {'name': 'BCEWithLogitsLoss', 'params': {}},
 'optimizer': {'name': 'Adam', 'params': {'lr': 0.001}},
 'scheduler': {'name': 'CosineAnnealingLR', 'params': {'T_max': 30}}}

In [63]:
def make_content_map_dict():
    questions_df = pd.read_csv(f'{const.INPUT_DATA_DIR}/questions.csv')
    q2p = dict(questions_df[['question_id', 'part']].values)
    q2p = np.array(list(q2p.values()))
    
    questions_df['tags'] = questions_df['tags'].fillna(0)
    questions_df['tag_list'] = questions_df['tags'].apply(lambda tags: [int(tag) for tag in str(tags).split(' ')])
    questions_df['tag_list'] = questions_df['tag_list'].apply(lambda x: [188] * (6 - len(x)) + x)
    q2tg = dict(questions_df[['question_id', 'tag_list']].values)
    q2tg = np.array(list(questions_df['tag_list'].values))
    return q2p, q2tg
q2p, q2tg = make_content_map_dict()

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import copy


# https://github.com/arshadshk/SAINT-pytorch/blob/main/saint.py
class Feed_Forward_block(nn.Module):
    """
    out =  Relu( M_out*w1 + b1) *w2 + b2
    """
    def __init__(self, dim_ff):
        super().__init__()
        self.layer1 = nn.Linear(in_features=dim_ff, out_features=dim_ff)
        self.layer2 = nn.Linear(in_features=dim_ff, out_features=dim_ff)

    def forward(self, ffn_in):
        return self.layer2(F.relu(self.layer1(ffn_in)))


class Encoder_block(nn.Module):
    """
    M = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    O = SkipConct(FFN(LayerNorm(M)))
    """

    def __init__(self, dim_model, heads_en, total_ex, total_cat, total_tg, seq_len):
        super().__init__()
        self.seq_len = seq_len - 1
        self.embd_ex = nn.Embedding(total_ex, embedding_dim=dim_model)
        self.embd_cat = nn.Embedding(total_cat + 1, embedding_dim=dim_model)
        self.embd_tg = nn.Embedding(total_tg + 1, embedding_dim=dim_model)
        self.embd_pos = nn.Embedding(seq_len, embedding_dim=dim_model)
        self.dt_fc = nn.Linear(1, dim_model, bias=False)
        # self.task_fc = nn.Linear(1, dim_model, bias=False)

        self.multi_en = nn.MultiheadAttention(embed_dim=dim_model, num_heads=heads_en)
        self.ffn_en = Feed_Forward_block(dim_model)
        self.layer_norm1 = nn.LayerNorm(dim_model)
        self.layer_norm2 = nn.LayerNorm(dim_model)

    def forward(self, in_ex, in_cat, in_tg, in_dt, first_block=True):
        device = in_ex.device

        if first_block:
            in_ex = self.embd_ex(in_ex)
            in_cat = self.embd_cat(in_cat)

            in_dt = in_dt.unsqueeze(-1)
            in_dt = self.dt_fc(in_dt)

            in_tg = self.embd_tg(in_tg)
            avg_in_tg_embed = in_tg.mean(dim=2)
            max_in_tg_embed = in_tg.max(dim=2).values

            # in_task = in_task.unsqueeze(-1)
            # in_task = self.task_fc(in_task)

            # combining the embedings
            # out = in_ex + in_cat + in_dt + (avg_in_tg_embed + max_in_tg_embed) + in_task
            out = in_ex + in_cat + in_dt + (avg_in_tg_embed + max_in_tg_embed)
        else:
            out = in_ex

        in_pos = get_pos(self.seq_len, device)
        in_pos = self.embd_pos(in_pos)
        out = out + in_pos

        out = out.permute(1, 0, 2)

        # Multihead attention
        n, _, _ = out.shape
        out = self.layer_norm1(out)
        skip_out = out
        out, attn_wt = self.multi_en(out, out, out,
                                     attn_mask=get_mask(seq_len=n, device=device))
        out = out + skip_out

        # feed forward
        out = out.permute(1, 0, 2)
        out = self.layer_norm2(out)
        skip_out = out
        out = self.ffn_en(out)
        out = out + skip_out

        return out


class Decoder_block(nn.Module):
    """
    M1 = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    M2 = SkipConct(Multihead(LayerNorm(M1;O;O)))
    L = SkipConct(FFN(LayerNorm(M2)))
    """
    def __init__(self, dim_model, total_in, total_exp, heads_de, seq_len):
        super().__init__()
        self.seq_len = seq_len - 1
        self.embd_in = nn.Embedding(total_in, embedding_dim=dim_model)
        # self.embd_exp = nn.Embedding(total_exp, embedding_dim=dim_model)
        self.embd_pos = nn.Embedding(self.seq_len, embedding_dim=dim_model)
        self.multi_de1 = nn.MultiheadAttention(embed_dim=dim_model, num_heads=heads_de)
        self.multi_de2 = nn.MultiheadAttention(embed_dim=dim_model, num_heads=heads_de)
        self.ffn_en = Feed_Forward_block(dim_model)
        self.el_fc = nn.Linear(1, dim_model, bias=False)

        self.layer_norm1 = nn.LayerNorm(dim_model)
        self.layer_norm2 = nn.LayerNorm(dim_model)
        self.layer_norm3 = nn.LayerNorm(dim_model)

    def forward(self, in_in, in_el, en_out, first_block=True):
        device = in_in.device

        if first_block:
            in_in = self.embd_in(in_in)

            in_el = in_el.unsqueeze(-1)
            in_el = self.el_fc(in_el)
            # in_exp = self.embd_exp(in_exp)

            # out = in_in + in_el + in_exp
            out = in_in + in_el
        else:
            out = in_in

        in_pos = get_pos(self.seq_len, device)
        in_pos = self.embd_pos(in_pos)
        out = out + in_pos

        out = out.permute(1, 0, 2)
        n, _, _ = out.shape

        out = self.layer_norm1(out)
        skip_out = out
        out, attn_wt = self.multi_de1(out, out, out,
                                      attn_mask=get_mask(seq_len=n, device=device))
        out = skip_out + out

        en_out = en_out.permute(1, 0, 2)
        en_out = self.layer_norm2(en_out)
        skip_out = out
        out, attn_wt = self.multi_de2(out, en_out, en_out,
                                      attn_mask=get_mask(seq_len=n, device=device))
        out = out + skip_out

        out = out.permute(1, 0, 2)
        out = self.layer_norm3(out)
        skip_out = out
        out = self.ffn_en(out)
        out = out + skip_out

        return out


def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def get_mask(seq_len, device):
    mask = torch.from_numpy(np.triu(np.ones((seq_len, seq_len)), k=1).astype(bool)).to(device)
    return mask


def get_pos(seq_len, device):
    # use sine positional embeddinds
    return torch.arange(seq_len, device=device).unsqueeze(0)


class SAINT(nn.Module):
    def __init__(self, dim_model, num_en, num_de, heads_en, total_ex, total_cat, total_tg, total_in, total_exp, heads_de, seq_len):
        super().__init__()

        self.num_en = num_en
        self.num_de = num_de

        self.encoder = get_clones(Encoder_block(dim_model, heads_en, total_ex, total_cat, total_tg, seq_len), num_en)
        self.decoder = get_clones(Decoder_block(dim_model, total_in, total_exp, heads_de, seq_len), num_de)

        self.out = nn.Linear(in_features=dim_model, out_features=1)

    def forward(self, feat):
        in_ex = feat['in_ex']
        in_dt = feat['in_dt']
        in_el = feat['in_el']
        in_tg = feat['in_tag']
        in_cat = feat['in_cat']
        in_in = feat['in_de']

        first_block = True
        for x in range(self.num_en):
            if x >= 1:
                first_block = False
            in_ex = self.encoder[x](in_ex, in_cat, in_tg, in_dt, first_block=first_block)
            in_cat = in_ex

        first_block = True
        for x in range(self.num_de):
            if x >= 1:
                first_block = False
            in_in = self.decoder[x](in_in, in_el, en_out=in_ex, first_block=first_block)

        # in_in = torch.sigmoid(self.out(in_in))
        in_in = self.out(in_in)

        return in_in.squeeze(-1)


model_encoder = {
    'transformer_saint_v6_2': SAINT,
}

def replace_fc(model, cfg):
    return model

class CustomModel(nn.Module):
    def __init__(self, cfg):
        super(CustomModel, self).__init__()
        self.cfg = cfg
        self.base_model = model_encoder[cfg['model']['backbone']](**cfg['model']['params'])
        self.model = replace_fc(self.base_model, cfg)
    def forward(self, x):
        x = self.model(x)
        return x

In [99]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomTestDataset7_2(Dataset):
    def __init__(self, df, q2p, q2tg, cfg=None):
        super(CustomTestDataset7_2, self).__init__()
        self.max_seq = cfg['params']['max_seq']
        self.n_skill = cfg['params']['n_skill']
        self.df = df
        self.q2p = q2p
        self.q2tg = q2tg
    def __len__(self):
        return len(self.df)
    def np_append(self, seq, val, dtype=int):
        new_seq = np.zeros(self.max_seq-1, dtype=dtype)
        new_seq[:-1] = seq[2:]
        new_seq[-1] = val
        return new_seq
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        row_id = row['row_id']
        uid = row["user_id"]
        cid = row["content_id"]
        timestamp = row["timestamp"]
        et = row["prior_question_elapsed_time"]
        cid_seq = np.zeros(self.max_seq, dtype=int)
        dt_seq = np.zeros(self.max_seq, dtype=int)
        et_seq = np.zeros(self.max_seq, dtype=int)
        qa_seq = np.zeros(self.max_seq, dtype=int)
        seq_len = 0
        # cid
        u_prev_cids = riiidFE.user_prev_ques_dict[uid]
        seq_len = len(u_prev_cids)
        print(seq_len)
        if seq_len > 0:     
            # difftime
            difftime = riiidFE.user_timestamp_dict[uid] + [timestamp]
            difftime = np.diff(difftime)
            # elapsed_time
            elapsedtime = riiidFE.user_et_dict[uid]
            if len(elapsedtime) != 0:
                # elapsedtime[0] = np.nan
                elapsedtime[0] = 0
            # answer_correctly
            qa = np.array(list(riiidFE.user_rec_dict[uid]), dtype=np.uint8)
            if seq_len >= self.max_seq:
                cid_seq = u_prev_cids[-self.max_seq:]
                dt_seq = difftime[-self.max_seq:]
                et_seq = elapsedtime[-self.max_seq:]
                qa_seq = qa[-self.max_seq:]
            else:
                cid_seq[-seq_len:] = u_prev_cids
                dt_seq[-seq_len:] = difftime
                et_seq[-seq_len:] = elapsedtime
                qa_seq[-seq_len:] = qa
        ## postprocess
        dt_seq = np.array(dt_seq) / 60_000.   # ms -> m
        dt_seq = np.where(dt_seq < 0, 300, dt_seq)
        dt_seq = np.log1p(dt_seq)[1:]
#         et_seq = np.append(et_seq[2:], [et])
        et_seq = self.np_append(et_seq, et, dtype=float)
        et_seq = np.array(et_seq) / 1_000.
        et_seq = np.log1p(et_seq)
        et_seq = np.where(np.isnan(et_seq), np.log1p(21), et_seq)
#         cid_seq = np.append(cid_seq[2:], [cid])
#         print(cid_seq)
        cid_seq = self.np_append(cid_seq, cid, dtype=int)
#         print(cid_seq)
#         part_seq = np.array([self.q2p[i] for i in cid_seq])  
        part_seq = self.q2p[cid_seq]
        qtg_seq = np.zeros((self.max_seq - 1, 6), dtype=int) + 188
        if seq_len > 0:
            qtg_seq[-seq_len:, :] = self.q2tg[cid_seq[-seq_len:]]
        else:
            qtg_seq[-1, :] = self.q2tg[cid_seq[-1]]
        qa_seq = qa_seq[1:]
        feat = {
            'in_ex': cid_seq,
            'in_dt': dt_seq,
            'in_el': et_seq,
            'in_tag': qtg_seq,
            'in_cat': part_seq,
            'in_de': qa_seq,
        }
        return feat

In [100]:
MAX_SEQ = cfg['data']['valid']['params']['max_seq'] - 1

def _test_epoch(model, test_dataset):
    model.eval()
    test_preds = np.zeros(test_dataset.__len__())
    
    bs = test_dataset.__len__()
    b_in_ex = np.zeros((bs, MAX_SEQ))
    b_in_dt = np.zeros((bs, MAX_SEQ))
    b_in_el = np.zeros((bs, MAX_SEQ))
    b_in_tag = np.zeros((bs, MAX_SEQ, 6))
    b_in_cat = np.zeros((bs, MAX_SEQ))
    b_in_de = np.zeros((bs, MAX_SEQ))
    for i in range(bs):
        b_in_ex[i] = test_dataset[i]['in_ex']
        b_in_dt[i] = test_dataset[i]['in_dt']
        b_in_el[i] = test_dataset[i]['in_el']
        b_in_tag[i] = test_dataset[i]['in_tag']
        b_in_cat[i] = test_dataset[i]['in_cat']
        b_in_de[i] = test_dataset[i]['in_de']

    feats = {
        'in_ex': torch.LongTensor(b_in_ex),
        'in_dt': torch.FloatTensor(b_in_dt),
        'in_el': torch.FloatTensor(b_in_el),
        'in_tag': torch.LongTensor(b_in_tag),
        'in_cat': torch.LongTensor(b_in_cat),
        'in_de': torch.LongTensor(b_in_de),    
    }
    with torch.no_grad():
        for k, v in feats.items():
            feats[k] = v.to(device)
        preds = model(feats)
        preds = preds[:, -1]
        test_preds = preds.sigmoid().cpu().detach().numpy()
    return test_preds

In [101]:
def prep_tags(x): return [int(i) for i in x.split()]
question = pd.read_csv(f'{const.INPUT_DATA_DIR}/questions.csv')
question['tags'] = question['tags'].fillna('0').apply(prep_tags)
question['part'] = question['part'].fillna(0).astype('int8')

In [102]:
iter_test = exp_test_df.groupby('group_num')

previous_test_df = None
for group_num, test_df in iter_test:
    if previous_test_df is not None:
        previous_test_df['answered_correctly'] = eval(test_df["prior_group_answers_correct"].iloc[0])
        previous_test_df['user_answer'] = eval(test_df["prior_group_responses"].iloc[0])
        riiidFE.add_user_feats(previous_test_df, add_feat=False, update_dict=True, val=True)
    
    test_df = pd.merge(test_df, question, left_on='content_id', right_on='question_id',  how="left")
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('int8')
    test_df['prior_question_elapsed_time'] = test_df['prior_question_elapsed_time'].fillna(-1).astype(int)
    
    test_df['part'] = test_df['part'].fillna(0.0).astype('int8')
    
    previous_test_df = test_df.copy()
    
    user_feat_df = riiidFE.add_user_feats(test_df, add_feat=True, update_dict=False, val=True)
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df = pd.concat([test_df, user_feat_df], axis=1)

    test_df = pd.concat([test_df.reset_index(drop=True),
                                   riiidFE.content_id_df.reindex(test_df['content_id'].values).reset_index(drop=True).iloc[:, 1:]], axis=1)
    
    test_dataset = CustomTestDataset7_2(test_df, q2p, q2tg, cfg['data']['valid'])
    
    break

100%|██████████| 18/18 [00:00<00:00, 1898.11it/s]


In [103]:
test_dataset[0].keys()

0


dict_keys(['in_ex', 'in_dt', 'in_el', 'in_tag', 'in_cat', 'in_de'])

In [104]:
test_dataset[0]['in_tag']

0


array([[188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 18

In [105]:
test_dataset[3]['in_tag']

120


array([[  0,   0, 143, 140,  81,  29],
       [  0,   0,   0,   0,   0, 125],
       [  0,   0,   0,   0,   0,  23],
       [  0,   0,   0,   0,   0, 180],
       [  0,   0,   0,   0,   0,  53],
       [  0,   0,   0,   0,   0,  43],
       [  0,   0,   0,   0,   0,   4],
       [  0,   0,   0,   0,   0,  33],
       [  0,   0,   0, 131,  93,  81],
       [  0,   0,   0,   0,   0,  54],
       [  0,   0,   0,   0,   0,  28],
       [  0,   0,   0,   0,   0, 134],
       [  0,   0,   0,   0,   0, 174],
       [  0,   0,   0,   0,   0, 179],
       [  0,   2, 107, 162,  81,  29],
       [  0,   0,   0,   0,   0,  53],
       [  0,   0,   0,   0,   0, 179],
       [  0,   0,   0,   0,   0,  52],
       [  0,   0,   0,   0,   0,  96],
       [  0,   0,   0,   0,   0, 115],
       [  0,   0,   0,   0,   0, 123],
       [  0,   0,   0,   0,   0, 170],
       [  0,   0,   0,   0,   0,  73],
       [  0,   0,   0,   0,   0, 168],
       [  0,   0,   0,   0,   0, 159],
       [  0, 143, 176,   

In [106]:
test_dataset[10]['in_tag']

80


array([[188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 188, 188, 188],
       [188, 188, 188, 18

In [ ]:
print(np.array(list(q2p.values()))[test_dataset[3]['in_ex']])

In [ ]:
test_dataset[3]['in_ex']

In [ ]:
test_df

In [ ]:
train_df[train_df['user_id'] == 288641214].tail(10)

In [ ]:
np.log1p((62798072960 - 62725820527) / 60_000.)